> # Betrachtung der Covid 19-Fälle in Deutschland

Die Datenbasis stellen die Dateien der Johns-Hopkins-Universität dar, die täglich einmal aktualisiert werden.

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [ ]:
def plot(plottype,dataframe, Title, Y_Lable, column):
    fig, ax = plt.subplots(figsize=(15,7))
    if plottype == 'line':
        ax.plot(dataframe['date'], dataframe[column], color='tab:blue', marker='o')
    elif plottype == 'bar':
        ax.bar(dataframe['date'], dataframe[column])
    ax.xaxis.set_major_locator(mdates.WeekdayLocator())
    ax.set(xlabel="Datum",
           ylabel= Y_Lable,           title = Title)
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
    plt.show()
    

In [ ]:
df = pd.read_csv("../input/corona-virus-report/covid_19_clean_complete.csv", parse_dates = ['Date'])

In [ ]:
df.info()

In [ ]:
print('Der erste Eintrag datiert vom:',df['Date'].iloc[0])
print('Der letzte Eintrag datiert vom:',df['Date'].iloc[-1])

In [ ]:
#Renaming the coulmns for easy usage
df.rename(columns={'Date': 'date', 
                     'Province/State':'state',
                     'Country/Region':'country',
                     'Lat':'lat', 'Long':'long',
                     'Confirmed': 'confirmed',
                     'Deaths':'deaths',
                     'Recovered':'recovered'
                    }, inplace=True)

df['active'] = df['confirmed'] - df['deaths'] - df['recovered']
df['mortality'] = df['deaths'] / df['confirmed'] * 100

In [ ]:
germany =  df[df.country == 'Germany']
germany = germany.groupby(by = 'date')['recovered', 'deaths', 'confirmed', 'active', 'mortality'].sum().reset_index()
germany

In [ ]:
plot('line',germany,'Gesamtsumme der bestätigten Fälle', '', 'confirmed')

Achtung! Bestätigte Fälle bedeutet für Deutschland: "Die durch Tests bestätigten Fälle". Die Anzahl der Corona-Infektionen ohne Test (= sogenannte Untererfassung) ist unbekannt. Die Anzahl der tatsächlich Infizierten inkl. Untererfassung ist aber auf jeden Fall höher als die hier dargestellten Zahlen.

**Update vom 26.03.2020**: Das Robert-Koch-Institut hat die Zählweise am 24.03.2020 geändert. Danach werden jetzt auch Fälle gezählt, wenn jemand Kontaktperson zu einer bestätigt, infizierten Person ist (s. https://www.berlin.de/corona/fallstatistik/). Damit lassen sich die Zahlen ab dem 24.03. nicht mehr mit den vorherigen vergleichen. Positiv: Die Dunkelziffer wird kleiner. Negativ: Es wird der Eindruck erweckt, dass sich jetzt mehr Menschen infizieren. 

**Update vom 27.03.2020**: Das Robert-Koch-Institut schätzt den Anteil der "Untererfassung" (s. Kapitel 7 https://www.rki.de/DE/Content/InfAZ/N/Neuartiges_Coronavirus/Steckbrief.html#doc13776792bodyText7) bei aller Vorsicht auf den Faktor 11 bis 20.

**Update vom 28.03.2020** Das bisher in Deutschland eingesetzte Testverfahren auf PCR-Basis, wurde bisher nicht validiert. Die Fehlerrate des Testverfahrens ist damit unbekannt. In China wurde ebenfalls ein PCR Verfahren eingesetzt. Die Fehlerrate nach Analyse eines chinesischen Forscherteams Anfang März lag bei mindestens 50% (s. Zhuang, G. H. et al. [Potential false-positive rate among the ‘asymptomatic infected individuals’ in close contacts of COVID-19 patients]. Zhonghua Liu Xing Bing Xue Za Zhi 41, 485–488 (2020))

In [ ]:
germany['new_confirmed'] = germany['confirmed'].shift(-1) - germany['confirmed'] 
plot('bar',germany,'Neu identifizierte Infektionen pro Tag', '', 'new_confirmed')

Der relativ starke Anstieg der täglich neuen Fälle ab dem 25.03.2020 könnte auf die Änderung der Zählweise zurückzuführen sein.

In [ ]:
plot('line',germany,'Summe der Toten', '', 'deaths')


Achtung! Corona-Todesfälle in Deutschland sind so definiert, dass beim Toten Corona-Viren nachgewiesen wurden. Ob der Verstorbene an Corona gestorben ist, oder an einer anderen Krankheit, ist damit nicht klar. Im Zweifel ist die Anzahl der tatsächlich ursächlich an Corona Verstorbenen geringer. 

In [ ]:
germany['new_deaths'] = germany['deaths'].shift(-1) - germany['deaths'] 
plot('bar',germany,'Neu Todesfälle pro Tag', '', 'new_deaths')

In [ ]:
print(f'Die Sterblichkeitsraten in Deutschland beträgt {germany["mortality"].iloc[-1]:.2f}%')

# Ableitung der erkrankten Fälle

Auf der basis der infizierten Fälle werden nun die Erkrankungen dargestellt:

* Keine Symptome 20% 
* Milde Fälle in 60%
* Schwere Symptome (z.B. Atemnot) in 14%
* Lebensbedrohlich 5%

Für die Ermittlung der benötigten Intensivbetten, wird jetzt die Anzahl der lebensbedrohlichen Fälle dargestellt. Da die Symptome nicht sofort bei Identifikation der Infektion lebensbedrohlich sind, wird mit einem Zeitverzug zwischen Identifikation und Behandlung auf Intensivstation mit +6 Tagen gerechnet. 

Die Behandlungsdauer in der Intesivstation liegt bei Corona-Erkrankten über 7 Tagen. Hier habe ich 10 Tage angenommen. Danach werden diese Betten wieder frei.

In [ ]:
lebensbedrohlich = 5 #in %
zeitdelta_lebensbedrohlich = 6
behandlungsdauer = 10
germany['lebensbedrohlich'] = germany['confirmed'].shift(zeitdelta_lebensbedrohlich) * lebensbedrohlich / 100
germany['benötigte Betten'] = germany['lebensbedrohlich'] - germany['lebensbedrohlich'].shift(behandlungsdauer) 
plot('line',germany,'Benötigte Intensivbetten', '', 'benötigte Betten')

lt. Melderegister DIVI liegt die Anzahl der behandelten Covid-Patienten in den Kliniken am 29.03.2020 07:00 Uhr bei 1.124 Patienten (s. https://www.divi.de/register/kartenansicht). Dies würde gut zu der dargestellten Kurve mit den dahinterliegenden Annahmen passen.

# Sterblichkeit je Altersklassen

Leider gibt es aktuell keine verlässlichen Daten zur Sterblickheit in Deutschland je Altersklassen. Auf der Suche nach Daten bin ich schließlich auf Süd-Korea gestoßen. Süd-Korea ist insofern interessant für Deutschland, da der Zustand des Gesundheitssystem in etwas auf dem Stand von Deutschland ist und dort genaue Daten zur Corona-Krise verfügbar sind.

Lt. Bericht von Professor Kim Woo-joo des Guro-Krankenhauses der Korea-Universität vom 28.03.2020 (https://www.youtube.com/watch?v=gAk7aX5hksU&t=526s) liegt die Sterbewahrscheinlichkeit für die unterschiedlichen Altersgruppen ins Südkorea bei folgenden Werten:

0-9 Jahre 0%, 10 - 19 Jahre 0%, 20 - 29 Jahre 0%, 30 - 39 Jahre 0,1%, 40 - 49 Jahre 0,1%, 50 - 59 Jahre 0,5%, 60 - 69 Jahre 1,6%, 70 - 79 Jahre 6.3%, ab 80 Jahre 11.6%

Aus der Bevölerkerungspyramide Deutschland werden die Anzahl Personen je Altersklasse dagegengesetzt (in Tausend):
0-9 Jahre 7760, 10-19 Jahre 7537, 20-29 Jahre 9573, 30-39 Jahre 10940, 40-49 Jahre 10105, 50-59 Jahre 13331, 60-69 Jahre 10741, 70 - 79 Jahre 7456, 80-89 Jahre 5074, 90-99 Jahre 832

Bei sofortiger uneingeschränkter Bewegungsfreiheit ohne besonderen Schutz der gesamten Bevölkerung würden sich folgende Todesfälle ergeben


In [ ]:
bev_altersklassen = [7760, 7537, 9573, 10940, 10105, 13331, 10741, 7456, 5074, 832, 0]
mort_altersklassen = [0, 0, 0, 0.1, 0.1, 0.5, 1.6, 6.3, 11.6, 11.6]
anz_sterbefälle = [x*y/100 for x,y in zip(bev_altersklassen, mort_altersklassen)]
_, ax = plt.subplots(figsize=(15,7))
ax.bar(range(10), anz_sterbefälle)
ax.set(xlabel="Altersklasen",title = 'Prognostizierte Todesfälle je Altersklassen')
for i, v in enumerate(anz_sterbefälle):
   ax.text(i-.25, v+5, f'{v*0.7*1000:,.0f}', fontsize=12, color='black')
plt.show()

# Strategie #1 Immunisierung der Gesellschaft

Die oben gezeigte maximale Anzahl von Todesfällen zeigt einen deutlichen Anstieg ab der Altersklasse 50 und älter an.

Ausgehend von diesen Zahlen könnte man folgende Strategie verfolgen:

* Jüngere Personen mit Alter bis zu 49 Jahren erhalten sofortigen Freigang ohne Einschränkungen
* Die Ansteckung dieser Personen mit dem Corona-Virus wird bewuss in Kauf genommen
* Ziel ist eine Immunisierung dieser Zielgruppe von ca 70% in ca. 4 Wochen
* Das Virus hat danach keine Möglichkeit sich weiter zu verbreiten
* Diese Zielgruppe muss den Kontakt zu den älternen Personen unbedingt vermeiden

Da die jüngeren Personen eine deutlich gerigere Gefahr tragen an dem Corona-Virus zu versterben, wird hier eine Abwägung zwischen dem Risiko zu erkranken bzw. zu sterben (bei dieser Altersgruppe liegt die Sterbewahrscheinlichkeit bei 0,1%) vs. der kompletten Abschottung einer Gesellschaft über einen längeren Zeitraum mit noch unabsehbaren Folgen vorgenommen.

Nach Ablauf der 4 Wochen könnte Schrittweise eine Lockerung der Kontaktbeschränkungen in den älteren Jahresgruppen erfolgen.
